In [2]:
import csv
raw_data = []
data= []
with open('datasets\car.csv') as file:
    csvreader = csv.reader(file)
    for rows in csvreader:
        data.append(rows)

In [19]:
data[0]

['buying_price',
 'maintenance_price',
 'number_doors',
 'capacity',
 'luggage_boot_size',
 'safety_level',
 'class']

In [34]:
#print out all the possible labels
labels = []
for _ in range(len(data[0])):
    labels.append([])
    
print(labels)

for rows in data[1:]:
    for i in range(len(rows)):
        if rows[i] not in labels[i]:
            labels[i].append(rows[i])
print(labels)

[[], [], [], [], [], [], []]
[['very_high', 'high', 'medium', 'low'], ['very_high', 'high', 'medium', 'low'], ['two', 'three', 'four', '5_or_more'], ['two', 'four', '5_or_more'], ['small', 'medium', 'large'], ['low', 'medium', 'high'], ['unacceptable', 'acceptable', 'very_good', 'good']]


0/1/2/3/4/5/6
for example its 3
3, 
[ 0, 1, 2, 3, 4]
[[],[],[],[],[]]

In [53]:
#total of 6 labels, split into 4 datasets, for each split more,... 

from math import log2
#get information gain
def info_gain(parent_data, split_index):
    
    split_label = labels[split_index]
    splits = []
    splited_child_entropy_sum = []
    for _ in range(len(split_label)):
        splits.append([])
        splited_child_entropy_sum.append([0,0,0,0])
    
    result_label_sum = [0,0,0,0]
    for rows in parent_data:
        #child entropy for each instances of the split
        for x in range(len(split_label)):
            if rows[split_index] == split_label[x]:
                splits[x].append(rows)
                for i in range(len(splited_child_entropy_sum)):
                    if rows[-1] == labels[-1][i]:
                        splited_child_entropy_sum[x][i] += 1
                        break
                break
            
        
        
        #parent entropy sum calculation
        for i in range(len(result_label_sum)):
            if rows[-1] == labels[-1][i]:
                result_label_sum[i] += 1
                break
            
    parent_entropy = 0
    total_values_in_parent_data = len(parent_data)
    for i in result_label_sum:
        parent_entropy += -1 * i / total_values_in_parent_data * log2(i/total_values_in_parent_data)
       
    # return splited_child_entropy_sum 
    #child entropy
    child_entropy = 0
    for n in range(len(split_label)):
        I = 0
        for m in splited_child_entropy_sum[n]:
            I += -1 * m / len(splits[n]) * log2(m / len(splits[n])) if m != 0 else 0
        child_entropy += len(splits[n])/len(parent_data)*I
            
    return parent_entropy - child_entropy, splits
